In [1]:
%load_ext autoreload
%autoreload 2

from glob import glob
import matplotlib.pyplot as plt 
import os
import pandas as pd

from mocca.campaign.model import HplcDadCampaign
from mocca.campaign.process_funcs import process_compound_experiments
from mocca.campaign.settings import Settings
from mocca.campaign.experiment import Experiment
from mocca.campaign.compound import Compound
from mocca.campaign.istd import InternalStandard

from mocca.dad_data.models import CompoundData, GradientData

from mocca.peak.models import CorrectedPeak
from mocca.peak.resolve_impure import parafac, get_parafac_peaks

In [2]:
# get the folders matching flag in directory
flag = '.txt'

###
# Add the path to the provided data folder
test_data_path = '/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data' # directory of test data
###

folders = glob(test_data_path + '/*' + flag + '*') #find all matching flag
folders = sorted(folders, key=lambda x: int(x.split('_')[-1][:-4]))

In [3]:
istd_folder = [folder for folder in folders if folder.split('_')[-1][:-4] == '96']

product_indices =['92', '93', '94', '95']
product_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in product_indices]

substrate_indices =['88', '89', '90', '91']
substrate_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in substrate_indices]

# obviously, we used a blank injection as the gradient run
gradient_folder = [folder for folder in folders if folder.split('_')[-1][:-4] == '97']

In [4]:
# remove some data which was gathered due to repeated start of the campaigns
bad_indices =['0', '1', '2', '3']
bad_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in bad_indices]

cnsource_indices =['98', '99']
cnsource_folders = [folder for folder in folders if folder.split('_')[-1][:-4] in cnsource_indices]

remove_folders = bad_folders + istd_folder + product_folders + substrate_folders + gradient_folder + cnsource_folders

reaction_folders = [folder for folder in folders if folder not in remove_folders]

In [5]:
gradient_experiment = Experiment(gradient_folder[0], gradient=True)

In [6]:
campaign = HplcDadCampaign('labsolutions')

In [7]:
campaign.settings

Settings(hplc_system_tag='labsolutions', detector_limit=2000, absorbance_threshold=None, wl_high_pass=None, wl_low_pass=None, peaks_high_pass=None, peaks_low_pass=None, spectrum_correl_thresh=None, relative_distance_thresh=None)

In [8]:
ArCl_concs = [0.0603, 0.04422, 0.03015, 0.01407]
ArCN_concs = [0.05955, 0.04367, 0.029775, 0.013895]
istd_conc = 0.06094

In [9]:
istd_experiment = Experiment(istd_folder[0], compound=Compound('tetralin', istd_conc, istd=True))

In [10]:
substrate_experiments = []
for i, conc in enumerate(ArCl_concs):
    exp = Experiment(substrate_folders[i], compound=Compound('2-chlorotuluene', conc), istd=InternalStandard('tetralin', istd_conc))
    substrate_experiments.append(exp)

In [11]:
product_experiments = []
for i, conc in enumerate(ArCN_concs):
    exp = Experiment(product_folders[i], compound=Compound('o-tolunitrile', conc), istd=InternalStandard('tetralin', istd_conc))
    product_experiments.append(exp)

In [12]:
campaign.add_experiment(gradient_experiment)
campaign.add_experiment(istd_experiment)
for exp in product_experiments:
    campaign.add_experiment(exp)
for exp in substrate_experiments:
    campaign.add_experiment(exp)
campaign.add_experiment(Experiment(reaction_folders[81]))

In [13]:
campaign.experiments

[Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_97.txt', compound=None, istd=None, gradient=True, processed=False),
 Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_96.txt', compound=Compound(key='tetralin', conc=0.06094, solvent=False, istd=True), istd=None, gradient=False, processed=False),
 Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_92.txt', compound=Compound(key='o-tolunitrile', conc=0.05955, solvent=False, istd=False), istd=[InternalStandard(key='tetralin', conc=0.06094)], gradient=False, processed=False),
 Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_93.txt', compound=Compound(key='o-tolunitrile', conc=0.04367, solvent=False, istd=False)

In [37]:
campaign.process_all_experiments(detector_limit=campaign.settings.detector_limit, wl_low_pass=None,
                                 spectrum_correl_thresh=0.9, relative_distance_thresh=0.01,
                                 wl_high_pass = 215, peaks_high_pass = 1, 
                                 peaks_low_pass = 5, absorbance_threshold = 700)

/Users/haascp/opt/anaconda3/envs/mocca-dev/lib/python3.9/site-packages/sklearn/metrics/_regression.py:781: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [38]:
for peak in campaign.chroms[0].peaks:
    print(peak.dataset.time[peak.maximum])
    print(peak.pure)
    print(peak.idx)


3.4548483838720427
True
-1
3.4548483838720427
False
1
3.5134955014995004
True
-1
3.753415528157281
True
2


In [39]:
for peak in campaign.peak_db:
    print(peak.compound_id)
    print(peak.idx)

tetralin
2
2-chlorotuluene
1
tetralin
2
2-chlorotuluene
1
tetralin
2
o-tolunitrile
1
tetralin
2
o-tolunitrile
1
tetralin
2
o-tolunitrile
1
tetralin
2
2-chlorotuluene
1
2-chlorotuluene
-1
None
1
unknown_1
-1
tetralin
2


In [40]:
template_peak = campaign.chroms[0].peaks[1]

In [41]:
peak = CorrectedPeak(left=template_peak.left,
                         right=template_peak.right,
                         maximum=template_peak.maximum,
                         dataset=template_peak.dataset,
                         idx=template_peak.idx,
                         saturation=template_peak.saturation,
                         pure=template_peak.pure,
                         integral=template_peak.integral,
                         offset=template_peak.offset,
                         istd=template_peak.istd)

In [42]:
peak

CorrectedPeak(left=1281, right=1378, maximum=1295, dataset=CompoundData(hplc_system_tag='labsolutions', experiment=Experiment(path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_85.txt', compound=None, istd=None, gradient=False, processed=True), path='/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_85.txt', data=array([[ 0.60540834,  0.60062571,  0.57864307, ..., -0.51211386,
        -0.51445048, -0.5151871 ],
       [ 0.53313182,  0.5290444 ,  0.51015698, ..., -0.43750861,
        -0.44044128, -0.44277396],
       [ 0.46866425,  0.46518778,  0.45011131, ..., -0.37197389,
        -0.37554289, -0.37911189],
       ...,
       [ 0.00985935,  0.00982639,  0.01159342, ...,  0.00576906,
         0.00699204,  0.00921501],
       [ 0.00773878,  0.00772346,  0.00730815, ...,  0.00359489,
         0.00421391,  0.00603293],
       [ 0.        ,  0.        ,  0

In [19]:
def overlap(peak, component):
        return component.left <= peak.left <= component.right \
            or peak.left <= component.left <= peak.right

In [20]:
for comp in campaign.quali_comp_db:
    print(comp.compound_id, comp.left, comp.right)
    print(overlap(peak, comp))

tetralin 1394 1459
False
2-chlorotuluene 1283 1326
True
o-tolunitrile 994 1091
False


In [21]:
parafac_peaks = get_parafac_peaks(peak, campaign.quali_comp_db, show_parafac_analytics = True)

In [22]:
peak.idx

1

In [23]:
parafac_peaks[0].idx

-1

In [231]:
parafac_tensor = (test[0], test[1], test[2])

[autoreload of mocca.peak.resolve_impure failed: Traceback (most recent call last):
  File "/Users/haascp/opt/anaconda3/envs/mocca-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/haascp/opt/anaconda3/envs/mocca-dev/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/Users/haascp/opt/anaconda3/envs/mocca-dev/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/haascp/opt/anaconda3/envs/mocca-dev/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen 

In [233]:
parafac_comp_tensors = []
for i in range(2):
    parafac_comp_tensor = (parafac_tensor[0][:, i],
                           parafac_tensor[1][:, i],
                           parafac_tensor[2][:, i])
    parafac_comp_tensors.append(parafac_comp_tensor)

In [248]:
parafac_comp_tensors[0][0].size

140

In [80]:
for peak in campaign.chroms[0].peaks:
    display(peak.maximum)
    display(peak.compound_id)
    display(peak.integral)
    display(peak.concentration)

376

'unknown_1'

206550.8402845089

None

751

...

None

1406

'tetralin'

45524.56221006884

0.030565292181505214

In [35]:
for chrom in campaign.compound_chroms:
    display(chrom.dataset.path)
    for peak in chrom.peaks:
        display(peak.compound_id)
        display(peak.integral)
        #display(peak.concentration)
        display(peak.istd)

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_96.txt'

'2-chlorotuluene'

10618.060848096951

[]

'tetralin'

...

101291.93802706029

[IstdPeak(maximum=1405, integral=39231.976970444244, offset=-1, compound_id='tetralin', concentration=0.06094)]

'tetralin'

39231.976970444244

[IstdPeak(maximum=1405, integral=39231.976970444244, offset=-1, compound_id='tetralin', concentration=0.06094)]

In [177]:
for comp in campaign.quali_comp_db.items:
    display(comp.compound_id)
    display(comp.maximum)
    display(len(comp.created_from))


'tetralin'

1406

7

'2-chlorotuluene'

1295

3

'o-tolunitrile'

1007

3

In [19]:
for comp in campaign.quant_comp_db.items:
    display(comp.compound_id)
    display(comp.calib_factors)
    display(comp.calib_data)
    display(comp.calib_scores)
    display(len(comp.created_from))

'tetralin'

{'absolute': 1489420.15472096}

{'absolute': [(0.06094, 90765.2642286953)]}

{'absolute': nan}

1

...

'o-tolunitrile'

{'absolute': 3237864.687574665, 'tetralin': 5.318045099264461}

{'absolute': [(0.05955, 203579.565272476),
  (0.04367, 123388.0305363853),
  (0.029775, 101291.93802706029)],
 'tetralin': [(0.05955, 0.3178851955827124),
  (0.04367, 0.23129423456953574),
  (0.029775, 0.15733927219674235)]}

{'absolute': 0.9199017630682809, 'tetralin': 0.9997418952808681}

3

In [65]:
for chrom in campaign.bad_chroms:
    display(chrom.dataset.path)

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_90.txt'

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_91.txt'

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_95.txt'

In [25]:
test = ((1, 2), (2, 4), (3, 6), (4, 8))

In [26]:
for i in zip(*test):
    print(i)

(1, 2, 3, 4)
(2, 4, 6, 8)


In [27]:
reaction_folders[81]

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_85.txt'

In [22]:
stats = campaign.quant_comp_db.items[2].calib_scores
max(stats, key=lambda key: stats[key])

'tetralin'

In [24]:
"a" is 'a'

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/c_/gt8zcwrn18s1dhd756dg2srw0000gn/T/ipykernel_1926/4094855256.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  "a" is 'a'


True

In [64]:
reaction_folders[0]

'/Users/haascp/Documents/PostDoc/MIT/MIC-DROP/cora/cyanation/wellplate_data/full plate combined data/09072021_sample_4.txt'

In [24]:
class Vehicle:
    pass

class Truck(Vehicle):
    pass

In [26]:
isinstance(Truck(), Vehicle)    # returns True

True